In [22]:

import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
print("gh")


gh


In [23]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
intents = json.loads(open('../AI_chatbot/sonuclar.json').read())
words=[]
classes=[]
documents=[]
ignore_letters=['?','!','.',',']

for intent in intents['sonuclar']:
    for pattern in intent['pattern']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)

[(['hosgeldin'], 'karsilama'), (['merhaba'], 'karsilama'), (['naber'], 'karsilama'), (['nasilsin'], 'karsilama'), (['nasil', 'gidiyor'], 'karsilama'), (['gule', 'gule'], 'gulegule'), (['iyi', 'gunler'], 'gulegule'), (['iyi', 'akÅŸamlar'], 'gulegule'), (['sonra', 'goruÅŸmek', 'uzere'], 'gulegule'), (['By', 'by'], 'gulegule'), (['kac', 'yas'], 'yas'), (['sen', 'kac', 'yasindasin'], 'yas'), (['yas', '?'], 'yas'), (['yas', 'kac'], 'yas'), (['yas', 'kac', 'oldu'], 'yas'), (['ismin', 'ne'], 'isim'), (['sana', 'nasil', 'seslenmeliyim'], 'isim'), (['isimii', 'oÄŸrenebilirmiim', '?'], 'isim'), (['sana', 'nasil', 'sesleniyorlar'], 'isim'), (['ne', 'satin', 'alabilirim', '?'], 'magaza'), (['ne', 'alabilirim', '?'], 'magaza'), (['onerin', 'ne', '?'], 'magaza'), (['ne', 'oneriyorsun', '?'], 'magaza'), (['hangi', 'urunler', 'var'], 'magaza'), (['calisma', 'saatleri'], 'zaman'), (['ne', 'zaman', 'sana', 'ulasabilirim'], 'zaman'), (['hep', 'burdamisin'], 'zaman')]


In [21]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))
pickle.dump(words,open("words.pkl",'wb'))
pickle.dump(classes,open('classes.pkl','wb'))


training =[]
output_empty=[0]*len(classes)

for document in documents:
    bag= []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        if word in word_patterns:
            bag.append(1) if word in word_patterns else bag.append(0)
        output_row = list(output_empty)
        output_row[classes.index(document[1])] = 1
        training.append([bag,output_row])

random.shuffle(training)
training = np.array(training,dtype=object)

train_x = list(training[:,0])
train_y = list(training[:,1])

model = Sequential()
model.add(Dense(128,input_shape= (len(train_x[0]),), activation="relu" ))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))

sgd = SGD(learning_rate=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss='categorical crossentropy', optimizer=sgd,metrics=['accuracy'])

model.fit(np.array(train_x, dtype=object), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('versiyon1.model')
print('Done')

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [25]:
training = []
output_empty = [0] * len( classes)
for doc in documents:
            bag = []
            word_patterns = doc[0]
            word_patterns = [ lemmatizer.lemmatize(word.lower()) for word in word_patterns]
            for word in  words:
                bag.append(1) if word in word_patterns else bag.append(0)

            output_row = list(output_empty)
            output_row[ classes.index(doc[1])] = 1
            training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('versiyon2.h5',hist)
print('Done')

Epoch 1/200


C:\Users\yilma\AppData\Local\Temp\ipykernel_10760\626678986.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)
c:\Users\yilma\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


6/6 [==============================] - 1s 2ms/step - loss: 1.8445 - accuracy: 0.1481
Epoch 2/200
6/6 [==============================] - 0s 3ms/step - loss: 1.7303 - accuracy: 0.3704
Epoch 3/200
6/6 [==============================] - 0s 3ms/step - loss: 1.5677 - accuracy: 0.4074
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 1.6078 - accuracy: 0.3704
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 1.3886 - accuracy: 0.4815
Epoch 6/200
6/6 [==============================] - 0s 3ms/step - loss: 1.5101 - accuracy: 0.4444
Epoch 7/200
6/6 [==============================] - 0s 5ms/step - loss: 1.3137 - accuracy: 0.4815
Epoch 8/200
6/6 [==============================] - 0s 3ms/step - loss: 1.2955 - accuracy: 0.5926
Epoch 9/200
6/6 [==============================] - 0s 4ms/step - loss: 1.1618 - accuracy: 0.5556
Epoch 10/200
6/6 [==============================] - 0s 4ms/step - loss: 1.1123 - accuracy: 0.5926
Epoch 11/200
6/6 [=======================